In [59]:
from sentence_transformers import SentenceTransformer, util
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize

import pandas as pd
import sqlite3
import numpy as np
import os

In [2]:
# load pretrained BERT model for phrase/sentence similarity
model = SentenceTransformer('all-MiniLM-L6-v2')

In [66]:

# generate map of symptom category (key) and corresponding list of search terms (values)
symptom_map = {
    "blood clot":["clot","blood clot","thrombosis"],
    "heart":["heart problem","heart issue","palpitation","rapid heartbeat","fast heartbeat", "increased heart rate"],
    "stroke":["stroke"],
    "dvt":["deep vein thrombosis"],
    "pulmonary embolism":["embolism","pulmonary embolism"],
    "breathing":["out of breath","shortness of breath","trouble breathing"],
    "lightheaded":["ligthheaded","lightheadedness","faint","dizzy","vertigo"],
    "leg":["leg pain","leg swelling"],
    "skin":["clammy skin","skin discoloration","cyanosis"],
}
# generate list of common self-report symptom phrases
self_report_phrases = ["I had <symptom>","I experienced <symptom>","I felt <symptom>","I suffer from",
"my symptoms included <symptom>","I felt <symptom>","dealing with <symptom>","has anyone else <symptom>"]

# create map between each search term (key) and the resultant self-report phrases and corresponding text embeddings
symptom_embedding_map = {}
for symptom_key in symptom_map:
    symptom_embedding_map[symptom_key] = []
    for symptom in symptom_map[symptom_key]:
        for phrase in self_report_phrases:
            phrase = phrase.replace("<symptom>",symptom)
            embedding = model.encode(phrase, convert_to_tensor=True)
            symptom_embedding_map[symptom_key].append([phrase,embedding])

In [23]:
# read in the data from sqlite DB
datadir = r"C:\Users\keatu\Regis_archive\practicum_data"
dbfile = os.path.join(datadir,"Facebook.db")
con = sqlite3.connect(dbfile)
posts = pd.read_sql("select * from posts",con)
comments = pd.read_sql("select * from comments", con)
replies = pd.read_sql("Select * from replies",con)
con.close()

# grab text/id fields from each data type--treating them all like unique posts
all_text = pd.concat([
                    posts[["user_id","post_id","text"]],
                    comments[["commenter_id","comment_id","comment_text"]].rename(columns={"commenter_id":"user_id","comment_id":"post_id","comment_text":"text"}),
                    replies[["commenter_id","comment_id","comment_text"]].rename(columns={"commenter_id":"user_id","comment_id":"post_id","comment_text":"text"})
                    ], sort = False)

In [72]:
len(all_text)

195682

In [82]:
# compare input sentence embedding to all matching sentence combinations and use threshold cosine similarity score
self_report_df = pd.DataFrame()
threshold = 0.75 # cosine similarity threshold
i=0
for idx, row in all_text.iterrows():
    i=+1
    if (i%10000)==0:
        print("{} completed of {}".format(i,len(all_text)))
    for sent in sent_tokenize(row['text']):
        sent_embed = model.encode(sent, convert_to_tensor=True)
        for term in symptom_embedding_map:
            top_score = 0
            top_match = ""
            for (match_sentence,match_embedding) in symptom_embedding_map[term]:
                cos_score = util.cos_sim(sent_embed, match_embedding).item()
                if cos_score > top_score:
                    top_score = cos_score
                    top_match = match_sentence
            if top_score > threshold:
                self_report_df = self_report_df.append({'user_id':row['user_id'],"post_id":row['post_id'],'sentence':sent,"match_sentence":top_match,"symptom":term,"cos_similarity":top_score}, ignore_index=True)

In [84]:
self_report_df.to_csv("self_report.csv")

In [5]:
self_reportdf = pd.read_csv("self_report.csv")

In [19]:
self_reportdf = self_reportdf.groupby(["post_id"]).aggregate("first").reset_index().drop(columns="Unnamed: 0").sort_values("user_id")

In [58]:
outcon = sqlite3.connect(r"C:\Users\keatu\Regis_archive\practicum_data\Facebook.db")
self_reportdf.astype(str).to_sql("self_reporting",con=outcon)
outcon.close()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\keatu\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-58-c6cd522d08ce>", line 2, in <module>
    self_reportdf.astype(str).to_sql("self_reporting",con=outcon)
  File "C:\Users\keatu\Anaconda3\lib\site-packages\pandas\core\generic.py", line 2712, in to_sql
    method=method,
  File "C:\Users\keatu\Anaconda3\lib\site-packages\pandas\io\sql.py", line 518, in to_sql
    method=method,
  File "C:\Users\keatu\Anaconda3\lib\site-packages\pandas\io\sql.py", line 1749, in to_sql
    table.create()
  File "C:\Users\keatu\Anaconda3\lib\site-packages\pandas\io\sql.py", line 644, in create
    "Table '{name}' already exists.".format(name=self.name)
ValueError: Table 'self_reporting' already exists.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\keatu\Anaconda3\

ValueError: Table 'self_reporting' already exists.

In [65]:
datadir = r"C:\Users\keatu\Regis_archive\practicum_data"
dbfile = os.path.join(datadir,"Facebook.db")
con = sqlite3.connect(dbfile)
self_reportdf = pd.read_sql("select * from self_reporting",con)
con.close()

In [66]:
user_ids = self_reportdf["user_id"].unique().tolist()

In [67]:
user_ids = [str(i) for i in user_ids]

In [68]:
sr_posts = posts[posts["user_id"].isin(user_ids)]
sr_comments = comments[comments["commenter_id"].isin(user_ids)]
sr_replies = replies[replies["commenter_id"].isin(user_ids)]

In [69]:
outcon = sqlite3.connect(r"C:\Users\keatu\Regis_archive\practicum_data\Facebook_Self_Report.db")
sr_posts.astype(str).to_sql("posts",con=outcon)
sr_comments.astype(str).to_sql("comments",con=outcon)
sr_replies.astype(str).to_sql("replies",con=outcon)
self_reportdf.astype(str).to_sql("self_reporting",con=outcon)
outcon.close()

In [57]:
user_ids

['2600187',
 '5136664',
 '6105165',
 '6600625',
 '7005237',
 '7910215',
 '7959932',
 '8323059',
 '10741906',
 '11319811',
 '12429554',
 '14211354',
 '14222379',
 '15721160',
 '16303420',
 '17506099',
 '19101360',
 '23108732',
 '31100485',
 '38415168',
 '38506246',
 '38900260',
 '40507055',
 '44000843',
 '48609823',
 '51907262',
 '57008075',
 '57201147',
 '58401286',
 '58900140',
 '90900699',
 '92503485',
 '153101028',
 '164901057',
 '207301137',
 '329900110',
 '500098950',
 '500213524',
 '504486561',
 '504654048',
 '505425078',
 '507212117',
 '510572378',
 '511100478',
 '512078296',
 '512218756',
 '513264295',
 '513950316',
 '514080966',
 '514285750',
 '519756930',
 '523462148',
 '526876071',
 '528781044',
 '530361076',
 '530450973',
 '530512498',
 '530653594',
 '533011117',
 '536107413',
 '536464283',
 '538159993',
 '538225265',
 '538350213',
 '539665264',
 '540934934',
 '542666175',
 '542858174',
 '542886923',
 '543139534',
 '544541528',
 '547622143',
 '553332791',
 '560328320',
 '56